In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, precision_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import StratifiedKFold
import numpy as np
import phik
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from helper import *
from plots import *
from category_encoders import TargetEncoder
from sklearn.preprocessing import FunctionTransformer
from imblearn.over_sampling import SMOTE
from functions import *
from imblearn.pipeline import Pipeline
import functions
import importlib
importlib.reload(functions)  # чтобы изменения в functions.py подхватились
import os

In [36]:
SEED = 42

In [37]:
#Cчитываем датасет
df = pd.read_csv(f"d:\dowload2\Australia_weather\weatherAUS.csv")
print(df.head())

         Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  Humidity3pm  \
0           W           44.0          W  ...        71.0         22.0   
1         WNW           44.0        NNW  ...        44.0         25.0   
2         WSW           46.0          W  ...        38.0         30.0   
3          NE           24.0         SE  ...        45.0         16.0   
4           W           41.0        ENE  ...        82.0         33.0   

   Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  RainToday  \
0       1007.7    

In [38]:
X, y, X_train, y_train, X_test, y_test, X_train_preprocessed, X_test_preprocessed = EDA_preprocess(df)

In [40]:
X_train_preprocessed.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,Location,WindGustDir,WindDir9am,WindDir3pm,Year,Month,Day
103642,19.300000,34.80000,0.0,56.0,15.0,7.0,52.000000,20.000000,1013.4,1010.6,24.300000,33.600000,0.0,0.200249,0.183750,0.148099,0.187714,2012.0,1.0,25.0
18697,12.172898,23.21887,0.2,39.0,11.0,22.0,68.838493,51.505227,1020.5,1019.0,16.977363,21.676257,0.0,0.269347,0.227687,0.263758,0.215530,2010.0,8.0,6.0
99305,13.000000,23.40000,0.0,43.0,2.0,22.0,64.000000,44.000000,1012.5,1010.4,16.400000,23.100000,0.0,0.223258,0.234448,0.280658,0.198091,2016.0,11.0,30.0
142981,25.000000,33.40000,0.0,63.0,19.0,26.0,75.000000,54.000000,1004.3,1001.2,29.400000,33.000000,0.0,0.171315,0.284763,0.306906,0.284914,2015.0,1.0,7.0
85572,15.300000,25.80000,0.0,30.0,9.0,13.0,58.000000,45.000000,1024.9,1021.2,21.300000,24.200000,0.0,0.224386,0.152037,0.159978,0.173299,2012.0,11.0,12.0


In [41]:
############################################### BASELINE DUMMY CLASSIFIER##########################################################
dummy_classfier = DummyClassifier(random_state=42, strategy='most_frequent')
dummy_classfier.fit(X_train_preprocessed, y_train)

y_pred = dummy_classfier.predict(X_test_preprocessed)
y_probs = dummy_classfier.predict_proba(X_test_preprocessed)[:, 1]

# Считаем метрики для модели и выводим графики
dummy_classifier_metrics = classification_metrics(
    y_test=y_test,
    y_pred=y_pred,
    y_probs=y_probs,
    model_name='DUMMY_CLASSIFIER'
)
print(dummy_classifier_metrics)

=== DUMMY_CLASSIFIER ===
Accuracy:  0.7758
Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000
ROC AUC:   0.5000
{'DUMMY_CLASSIFIER_Accuracy': 0.775835999859348, 'DUMMY_CLASSIFIER_Precision': 0.0, 'DUMMY_CLASSIFIER_Recall': 0.0, 'DUMMY_CLASSIFIER_F1': 0.0, 'DUMMY_CLASSIFIER_ROC_AUC': np.float64(0.5)}


c:\Users\dima2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [42]:
############################################### BASELINE LOG_REG CLASSIFIER##########################################################
LogReg_classfier = LogisticRegression(random_state=42)
LogReg_classfier.fit(X_train_preprocessed, y_train)

y_pred = LogReg_classfier.predict(X_test_preprocessed)
y_probs = LogReg_classfier.predict_proba(X_test_preprocessed)[:, 1]

# Считаем метрики для модели и выводим графики
LogReg_classifier_metrics = classification_metrics(
    y_test=y_test,
    y_pred=y_pred,
    y_probs=y_probs,
    model_name='LOG_REG_CLASSIFIER'
)
print(LogReg_classifier_metrics)

=== LOG_REG_CLASSIFIER ===
Accuracy:  0.8397
Precision: 0.7263
Recall:    0.4569
F1 Score:  0.5609
ROC AUC:   0.8508
{'LOG_REG_CLASSIFIER_Accuracy': 0.8396568093111572, 'LOG_REG_CLASSIFIER_Precision': 0.7262528047868362, 'LOG_REG_CLASSIFIER_Recall': 0.45694117647058824, 'LOG_REG_CLASSIFIER_F1': 0.560947429231658, 'LOG_REG_CLASSIFIER_ROC_AUC': np.float64(0.8508383241621523)}


c:\Users\dima2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
############################################### BASELINE TREE CLASSIFIER##########################################################
Tree_classfier = DecisionTreeClassifier(random_state=42, max_depth=10)
Tree_classfier.fit(X_train_preprocessed, y_train)

y_pred = Tree_classfier.predict(X_test_preprocessed)
y_probs = Tree_classfier.predict_proba(X_test_preprocessed)[:, 1]

# Считаем метрики для модели и выводим графики
Tree_classifier_metrics = classification_metrics(
    y_test=y_test,
    y_pred=y_pred,
    y_probs=y_probs,
    model_name='TREE_CLASSIFIER'
)
print(Tree_classifier_metrics)

=== TREE_CLASSIFIER ===
Accuracy:  0.8377
Precision: 0.6982
Recall:    0.4860
F1 Score:  0.5731
ROC AUC:   0.8409
{'TREE_CLASSIFIER_Accuracy': 0.8376876824079609, 'TREE_CLASSIFIER_Precision': 0.6982195176921343, 'TREE_CLASSIFIER_Recall': 0.48596078431372547, 'TREE_CLASSIFIER_F1': 0.5730669626341103, 'TREE_CLASSIFIER_ROC_AUC': np.float64(0.8408902088754284)}


In [44]:
importances = pd.DataFrame({
    'Feature': X_train_preprocessed.columns,
    'Importance': Tree_classfier.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(importances)

          Feature  Importance
7     Humidity3pm    0.562221
2        Rainfall    0.091169
3   WindGustSpeed    0.089302
9     Pressure3pm    0.067386
13       Location    0.025699
0         MinTemp    0.024725
16     WindDir3pm    0.019751
6     Humidity9am    0.019645
11        Temp3pm    0.014665
1         MaxTemp    0.014187
8     Pressure9am    0.010752
5    WindSpeed3pm    0.010690
10        Temp9am    0.010307
15     WindDir9am    0.008052
4    WindSpeed9am    0.006418
14    WindGustDir    0.005767
18          Month    0.005646
12      RainToday    0.005504
19            Day    0.004657
17           Year    0.003456


In [45]:
############################################### BASELINE RANDOM_FOREST CLASSIFIER##########################################################
random_forest_classfier = RandomForestClassifier(random_state=42)
random_forest_classfier.fit(X_train_preprocessed, y_train)

y_pred = random_forest_classfier.predict(X_test_preprocessed)
y_probs = random_forest_classfier.predict_proba(X_test_preprocessed)[:, 1]

# Считаем метрики для модели и выводим графики
random_forest_classifier_metrics = classification_metrics(
    y_test=y_test,
    y_pred=y_pred,
    y_probs=y_probs,
    model_name='RANDOM_FOREST_CLASSIFIER'
)
print(random_forest_classifier_metrics)

=== RANDOM_FOREST_CLASSIFIER ===
Accuracy:  0.8569
Precision: 0.7745
Recall:    0.5098
F1 Score:  0.6149
ROC AUC:   0.8850
{'RANDOM_FOREST_CLASSIFIER_Accuracy': 0.8568515067337108, 'RANDOM_FOREST_CLASSIFIER_Precision': 0.7745471877979028, 'RANDOM_FOREST_CLASSIFIER_Recall': 0.5098039215686274, 'RANDOM_FOREST_CLASSIFIER_F1': 0.614889792829439, 'RANDOM_FOREST_CLASSIFIER_ROC_AUC': np.float64(0.8849865133870806)}


In [46]:
importances = pd.DataFrame({
    'Feature': X_train_preprocessed.columns,
    'Importance': random_forest_classfier.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(importances)

          Feature  Importance
7     Humidity3pm    0.190457
9     Pressure3pm    0.063838
6     Humidity9am    0.061327
2        Rainfall    0.059911
8     Pressure9am    0.057610
3   WindGustSpeed    0.057088
11        Temp3pm    0.051542
0         MinTemp    0.049851
1         MaxTemp    0.047176
10        Temp9am    0.046286
13       Location    0.041756
19            Day    0.037351
5    WindSpeed3pm    0.034849
4    WindSpeed9am    0.032411
16     WindDir3pm    0.031206
15     WindDir9am    0.029730
14    WindGustDir    0.029454
17           Year    0.026492
18          Month    0.026457
12      RainToday    0.025209


In [47]:
############################################### BASELINE GRADIENT_BOOSTING CLASSIFIER##########################################################
gradient_boosting_classfier =GradientBoostingClassifier(random_state=42)
gradient_boosting_classfier.fit(X_train_preprocessed, y_train)

y_pred =gradient_boosting_classfier.predict(X_test_preprocessed)
y_probs = gradient_boosting_classfier.predict_proba(X_test_preprocessed)[:, 1]

# Считаем метрики для модели и выводим графики
gradient_boosting_classifier_metrics = classification_metrics(
    y_test=y_test,
    y_pred=y_pred,
    y_probs=y_probs,
    model_name='GRADIENT_BOOSTING_CLASSIFIER'
)
print(gradient_boosting_classifier_metrics)

=== GRADIENT_BOOSTING_CLASSIFIER ===
Accuracy:  0.8498
Precision: 0.7511
Recall:    0.4936
F1 Score:  0.5957
ROC AUC:   0.8692
{'GRADIENT_BOOSTING_CLASSIFIER_Accuracy': 0.8498189106508668, 'GRADIENT_BOOSTING_CLASSIFIER_Precision': 0.7510739856801909, 'GRADIENT_BOOSTING_CLASSIFIER_Recall': 0.49364705882352944, 'GRADIENT_BOOSTING_CLASSIFIER_F1': 0.595740653099858, 'GRADIENT_BOOSTING_CLASSIFIER_ROC_AUC': np.float64(0.8692466941091157)}


In [48]:
importances = pd.DataFrame({
    'Feature': X_train_preprocessed.columns,
    'Importance': gradient_boosting_classfier.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(importances)

          Feature  Importance
7     Humidity3pm    0.644552
2        Rainfall    0.091723
3   WindGustSpeed    0.086600
9     Pressure3pm    0.078729
13       Location    0.018632
16     WindDir3pm    0.018118
11        Temp3pm    0.012508
0         MinTemp    0.009725
6     Humidity9am    0.009255
10        Temp9am    0.007291
12      RainToday    0.005691
1         MaxTemp    0.005421
5    WindSpeed3pm    0.004886
18          Month    0.001950
15     WindDir9am    0.001699
4    WindSpeed9am    0.000913
14    WindGustDir    0.000895
17           Year    0.000870
8     Pressure9am    0.000539
19            Day    0.000000
